Exploring several features

In [27]:
from abc import ABC, abstractmethod
import re
from sacremoses import MosesTokenizer
import Levenshtein
import spacy
import nltk
import pickle
import urllib
import os
import tarfile
import zipfile
import seaborn as sns
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import numpy as np
# import paths
from string import punctuation
import matplotlib.pyplot as plt
nltk.download('stopwords', quiet=True)
from nltk.corpus import stopwords
from nltk import word_tokenize
RESOURCES_DIR = Path('../resources')
DATASETS_PATH = RESOURCES_DIR / "datasets"
WORD_EMBEDDINGS_NAME = "model" #  "glove.42B.300d"  #  #'glove.6B.300d'
DUMPS_DIR = RESOURCES_DIR / "DUMPS"
import requests
import gensim

stopwords = set(stopwords.words("dutch"))


In [28]:
# alternatives for Glove in Dutch: 

# https://fasttext.cc/docs/en/support.html#building-fasttext-python-module  voc size unknown, dims 300 installation failed because of wheel.
# http://vectors.nlpl.eu/repository/#  voc size 2610658, dims 100
# coosto https://github.com/coosto/dutch-word-embeddings  vocabulary 250479 dims 300 media data! 

In [40]:
def ControlDivisionByZero(numerator, denominator):
    return numerator / denominator if denominator != 0 else 0


class FeatureAbstract(ABC):

    @abstractmethod
    def get_ratio(self, kwargs):
        pass

    @abstractmethod
    def calculate_ratio(self, simple_text, original_text):
        pass


class Feature(FeatureAbstract):

    def __init__(self, split, target_ratio):
        self.split = split
        self.target_ratio = target_ratio

    def get_ratio(self, kwargs):
        if not 'original_text_preprocessed' in kwargs:
            kwargs['original_text_preprocessed'] = ""

        if self.split == "train":
            simple_text = kwargs.get('simple_text')
            original_text = kwargs.get('original_text')
            result_ratio = self.calculate_ratio(simple_text, original_text)

        elif self.split == "valid" or self.split == "test":
            result_ratio = self.target_ratio
        else:
            raise ValueError("stage value not supported")
        kwargs['original_text_preprocessed'] += f'{self.name}_{result_ratio} '
        return kwargs

    @property
    def name(self):
        class_name = self.__class__.__name__
        name = ""
        for word in re.findall('[A-Z][^A-Z]*', class_name):
            if word: name += word[0]
        if not name: name = class_name
        return name


class WordLengthRatio(Feature):

    def __init__(self, stage, target_ratio):
        super().__init__(stage, target_ratio)
        if stage == "train":
            # THIS IS A WORD TOKENIZER, we need one for dutch
            # nl_core_news_sm spacy  spacy.nl_core_news_sm 
            # nltk.word_tokenize
            self.tokenizer =  MosesTokenizer(lang='nl') #  nltk.word_tokenize(language='dutch')  # Moses Tokenizer for Dutch language

    def calculate_ratio(self, simple_text, original_text):
        return round(ControlDivisionByZero(
            len(self.tokenizer.tokenize(simple_text)),
            len(self.tokenizer.tokenize(original_text))), 2)


class CharLengthRatio(Feature):

    def __init__(self, stage, target_ratio):
        super().__init__(stage, target_ratio)

    def calculate_ratio(self, simple_text, original_text):
        return round(ControlDivisionByZero(len(simple_text),
            len(original_text)), 2)


class LevenshteinRatio(Feature):

    def __init__(self, stage, target_ratio):
        super().__init__(stage, target_ratio)

    def calculate_ratio(self, simple_text, original_text):
        simple_text = word_tokenize(simple_text,language='dutch')
        original_text = word_tokenize(original_text,language='dutch')
        return round(Levenshtein.seqratio(original_text,
                                       simple_text), 2)


class DependencyTreeDepthRatio(Feature):

    def __init__(self, stage, target_ratio):
        super().__init__(stage, target_ratio)
        if stage == "train":
            self.nlp = self.get_spacy_model()

    def get_spacy_model(self):

        model = 'nl_core_news_sm'  # from spacy, Dutch pipeline optimized for CPU. Components: tok2vec, morphologizer, tagger, parser, lemmatizer (trainable_lemmatizer), senter, ner.
        if not spacy.util.is_package(model):
            spacy.cli.download(model)
            spacy.cli.link(model, model, force=True, model_path=spacy.util.get_package_path(model))
        return spacy.load(model)

    def calculate_ratio(self, simple_text, original_text):

        result_ratio = round(ControlDivisionByZero(
            self.get_dependency_tree_depth(simple_text),
            self.get_dependency_tree_depth(original_text)), 2)

        return result_ratio

    def get_dependency_tree_depth(self, sentence):

        def get_subtree_depth(node):
            if len(list(node.children)) == 0:
                return 0
            return 1 + max([get_subtree_depth(child) for child in node.children])

        tree_depths = [get_subtree_depth(spacy_sentence.root) for spacy_sentence in self.nlp(sentence).sents]
        if len(tree_depths) == 0:
            return 0
        return max(tree_depths)


class WordRankRatio(Feature):
    # single underscore = internally

    def __init__(self, stage, target_ratio): # constructor of the class 
        super().__init__(stage, target_ratio)
        if stage == "train":
            self.tokenizer = MosesTokenizer(lang='nl')
            self.word2rank = self._get_word2rank()
            print('this is word2rank', self.word2rank)
            print('length of word2rank', len(self.word2rank))
            self.length_rank = len(self.word2rank) # hier length of the file! 

    def calculate_ratio(self, simple_text, original_text):

        result_ratio = round(min(ControlDivisionByZero(self.get_lexical_complexity_score(simple_text),
                                                       self.get_lexical_complexity_score(original_text)),
                                 2), 2)

        return result_ratio

    def get_lexical_complexity_score(self, sentence, quantile_value=0.75):

        words = self.tokenizer.tokenize(self._remove_stopwords(self._remove_punctuation(sentence)))
        words = [word for word in words if word in self.word2rank]
        if len(words) == 0:
            return np.log(1 + self.length_rank)
        return np.quantile([self._get_rank(word) for word in words], quantile_value)

    def _remove_punctuation(self, text):
        return ' '.join([word for word in self.tokenizer.tokenize(text) if not self._is_punctuation(word)])

    def _remove_stopwords(self, text):
        return ' '.join([w for w in self.tokenizer.tokenize(text) if w.lower() not in stopwords])

    def _is_punctuation(self, word):
        return ''.join([char for char in word if char not in punctuation]) == ''

    def _get_rank(self, word):
        rank = self.word2rank.get(word, self.length_rank)
        return np.log(1 + rank)

    def _get_word2rank(self, vocab_size=np.inf):
        model_filepath = DUMPS_DIR / f"{WORD_EMBEDDINGS_NAME}.pk"
        if model_filepath.exists():
            with open(model_filepath, 'rb') as f:
                model = pickle.load(f)
            return model
        
        
        else:
            # print("Downloading glove ...") # pretrained vectors
            # self._download_glove(model_name='glove.6B', dest_dir=str(DUMPS_DIR))
            # print("Preprocessing word2rank...")
            # DUMPS_DIR.mkdir(parents=True, exist_ok=True)
            # WORD_EMBEDDINGS_PATH = DUMPS_DIR / f'{WORD_EMBEDDINGS_NAME}.txt'
            # lines_generator = self._yield_lines(WORD_EMBEDDINGS_PATH)
            # word2rank = {}
            # # next(lines_generator)
            # print('vocab_size', vocab_size)
            # for i, line in enumerate(lines_generator):
            #     if i >= vocab_size: break
            #     word = line.split(' ')[0]
            #     print('word', word)
            #     word2rank[word] = i
            #     print('ranked word?', word2rank[word])

            # pickle.dump(word2rank, open(model_filepath, 'wb'))
            # txt_file = DUMPS_DIR / f'{WORD_EMBEDDINGS_NAME}.txt'
            # zip_file = DUMPS_DIR / f'{WORD_EMBEDDINGS_NAME}.zip'
            # # if txt_file.exists(): txt_file.unlink()
            # if zip_file.exists(): zip_file.unlink()
            # print(word2rank)
            # return word2rank
            
            print("Downloading dutch embeddings ...") # pretrained vectors
            self._download_embeddings(model_name='coostco', dest_dir=str(DUMPS_DIR))
            print("Preprocessing word2rank...")
            DUMPS_DIR.mkdir(parents=True, exist_ok=True)
            WORD_EMBEDDINGS_PATH = DUMPS_DIR / f'{WORD_EMBEDDINGS_NAME}.bin'
            model = self._load_word_embeddings(WORD_EMBEDDINGS_PATH) # returns index_to_key
            # store into file
            lines_generator = self._yield_lines(model) # (WORD_EMBEDDINGS_PATH)
            
            word2rank = {}
            # next(lines_generator)
            print('vocab_size', vocab_size)
            for i, line in enumerate(lines_generator):
                if i >= vocab_size: break # its not vocab size any more but  # len(model.key_to_index)
                word = line.split(',')[0]
                # print('word', word)
                word2rank[word] = i
                # print('ranked word?', word2rank[word])
                
            pickle.dump(word2rank, open(model_filepath, 'wb'))
            txt_file = DUMPS_DIR / f'{WORD_EMBEDDINGS_NAME}.txt'
            zip_file = DUMPS_DIR / f'{WORD_EMBEDDINGS_NAME}.zip'
            # if txt_file.exists(): txt_file.unlink()
            # if zip_file.exists(): zip_file.unlink()
            # print(word2rank)
            return word2rank
    
    
            #     WE_FILE = 'coosto.bin'
        # wordembeddings_url = 'https://github.com/coosto/dutch-word-embeddings/releases/download/v1.0/model.bin'
        # r = requests.get(wordembeddings_url)
        # if r.status_code == 200:
        #     with open(WE_FILE, 'wb') as wordembeddings_file:
        #         wordembeddings_file.write(r.content)
            
        # https://github.com/tommasoc80/DALC/blob/2085fddf0fbac065c493b97079ae4ec27ddc7e00/v2.0/models/.ipynb_checkpoints/DALC_BiLSTM_offensive_binary-checkpoint.ipynb
    
                # store as TXT? 
                # download_word_embeddings()
    
    def _load_word_embeddings(self, filepath):
        model = gensim.models.KeyedVectors.load_word2vec_format(filepath, binary=True) # '../resources/DUMPS/model.bin'
        model_indexes = model.index_to_key
        return model_indexes

#  word_model = load_word_embeddings()
    # def _vectorize_data(data, wm):
    #     # turn the tokens into coosto vocab indices
    #     # these will be converted to embeddings in the Embedding layer
    #     vocab = wm.vocab
    #     keys = list(vocab.keys())
    #     final = []
    #     for tweet in data:
    #         final.append([keys.index(word) for word in tweet if vocab.get(word, None) is not None])
    #     return final

    def _download_glove(self, model_name, dest_dir): # pretrained rankings
        url = ''
        if model_name == 'glove.6B':
            url = 'http://nlp.stanford.edu/data/glove.6B.zip'
        elif model_name == 'glove.42B.300d':
            url = 'http://nlp.stanford.edu/data/glove.42B.300d.zip'
        elif model_name == 'glove.840B.300d':
            url = 'http://nlp.stanford.edu/data/glove.840B.300d.zip'
        elif model_name == 'glove.twitter.27B':
            url = 'http://nlp.stanford.edu/data/glove.twitter.27B.zip',
        else:
            possible_values = ['glove.6B.300d', 'glove.42B.300d', 'glove.840B.300d', 'glove.twitter.27B']
            raise ValueError('Unknown model_name. Possible values are {}'.format(possible_values))
        file_path = self._download_url(url, dest_dir)
        out_filepath = Path(file_path)
        out_filepath = out_filepath.parent / f'{out_filepath.stem}.txt'
        # print(out_filepath, out_filepath.exists())
        if not out_filepath.exists():
            print("Extracting: ", Path(file_path).name)
            self._unzip(file_path, dest_dir)
    
    def _download_embeddings(self, model_name, dest_dir): # pretrained rankings
        url = ''
        if model_name == 'coostco':
            url = 'https://github.com/coosto/dutch-word-embeddings/releases/download/v1.0/model.bin'

        file_path = self._download_url(url, dest_dir)
        out_filepath = Path(file_path)
        out_filepath = out_filepath.parent / f'{out_filepath.stem}.txt'
        # print(out_filepath, out_filepath.exists())
        if not out_filepath.exists():
            print("Extracting: ", Path(file_path).name)
            self._unzip(file_path, dest_dir)

    def _yield_lines(self, filepath):
        filepath = Path(filepath)
        with filepath.open('r', encoding="latin-1") as f:
            for line in f:
                yield line.rstrip()

    def _download_url(self, url, output_path):
        name = url.split('/')[-1]
        file_path = f'{output_path}/{name}'
        if not Path(file_path).exists():
            with tqdm(unit='B', unit_scale=True, leave=True, miniters=1,
                      desc=name) as t:  # all optional kwargs
                urllib.request.urlretrieve(url, filename=file_path, reporthook=self._download_report_hook(t), data=None)
        return file_path

    def _unzip(self, file_path, dest_dir=None):
        if dest_dir is None:
            dest_dir = os.path.dirname(file_path)
        if file_path.endswith('.zip'):
            with zipfile.ZipFile(file_path, "r") as zip_ref:
                zip_ref.extractall(dest_dir)
        elif file_path.endswith("tar.gz") or file_path.endswith("tgz"):
            tar = tarfile.open(file_path, "r:gz")
            tar.extractall(dest_dir)
            tar.close()
        elif file_path.endswith("tar"):
            tar = tarfile.open(file_path, "r:")
            tar.extractall(dest_dir)
            tar.close()

    def _download_report_hook(self, t):
        last_b = [0]

        def inner(b=1, bsize=1, tsize=None):
            if tsize is not None:
                t.total = tsize
            t.update((b - last_b[0]) * bsize)
            last_b[0] = b

        return inner

In [41]:
sentences_pair = {"simple_text": "Dit is de complexe versie", "original_text": "Dit is de vereenvoudigte versie."}

In [42]:
word_rank = WordRankRatio("train", 0.8)
sentences_pair = word_rank.get_ratio(sentences_pair)
sentences_pair

Extracting:  model.bin
Preprocessing word2rank...
vocab_size inf


In [ ]:
# model = gensim.models.KeyedVectors.load_word2vec_format('../resources/DUMPS/model.bin', binary=True)

FileNotFoundError: [Errno 2] No such file or directory: '../resources/DUMPS/model.bin'

In [ ]:
# model.key_to_index

{'</s>': 0,
 'de': 1,
 '<mention>': 2,
 '<url>': 3,
 'een': 4,
 'het': 5,
 'en': 6,
 'van': 7,
 'in': 8,
 'ik': 9,
 'is': 10,
 'op': 11,
 'dat': 12,
 'je': 13,
 'voor': 14,
 'met': 15,
 'niet': 16,
 'te': 17,
 'zijn': 18,
 'die': 19,
 'maar': 20,
 'ook': 21,
 'er': 22,
 'als': 23,
 'om': 24,
 'bij': 25,
 'aan': 26,
 'rt': 27,
 'dan': 28,
 'wat': 29,
 'nog': 30,
 'wel': 31,
 'ze': 32,
 'dit': 33,
 'naar': 34,
 'of': 35,
 'we': 36,
 'over': 37,
 'heb': 38,
 'al': 39,
 'kan': 40,
 'heeft': 41,
 'zo': 42,
 'uit': 43,
 'mijn': 44,
 'nu': 45,
 'meer': 46,
 'door': 47,
 'was': 48,
 'deze': 49,
 'geen': 50,
 'hij': 51,
 'hebben': 52,
 'weer': 53,
 'ben': 54,
 'moet': 55,
 'goed': 56,
 'wordt': 57,
 'worden': 58,
 'dus': 59,
 'gaat': 60,
 'toch': 61,
 'veel': 62,
 'echt': 63,
 'gaan': 64,
 'zou': 65,
 'kunnen': 66,
 'na': 67,
 'daar': 68,
 'wil': 69,
 'hoe': 70,
 'via': 71,
 'hier': 72,
 'vind': 73,
 'mensen': 74,
 'doen': 75,
 'nieuwe': 76,
 'waar': 77,
 'tot': 78,
 'jaar': 79,
 'eens': 80,
 '

In [ ]:
# len(model.key_to_index)

250479

In [ ]:
# we just need the list of words. end. 
# model.index_to_key

['</s>',
 'de',
 '<mention>',
 '<url>',
 'een',
 'het',
 'en',
 'van',
 'in',
 'ik',
 'is',
 'op',
 'dat',
 'je',
 'voor',
 'met',
 'niet',
 'te',
 'zijn',
 'die',
 'maar',
 'ook',
 'er',
 'als',
 'om',
 'bij',
 'aan',
 'rt',
 'dan',
 'wat',
 'nog',
 'wel',
 'ze',
 'dit',
 'naar',
 'of',
 'we',
 'over',
 'heb',
 'al',
 'kan',
 'heeft',
 'zo',
 'uit',
 'mijn',
 'nu',
 'meer',
 'door',
 'was',
 'deze',
 'geen',
 'hij',
 'hebben',
 'weer',
 'ben',
 'moet',
 'goed',
 'wordt',
 'worden',
 'dus',
 'gaat',
 'toch',
 'veel',
 'echt',
 'gaan',
 'zou',
 'kunnen',
 'na',
 'daar',
 'wil',
 'hoe',
 'via',
 'hier',
 'vind',
 'mensen',
 'doen',
 'nieuwe',
 'waar',
 'tot',
 'jaar',
 'eens',
 'mij',
 'haar',
 'heel',
 'gewoon',
 'me',
 'alleen',
 'had',
 'maken',
 'mee',
 'tegen',
 'hun',
 'jij',
 'komt',
 'even',
 '1',
 'wij',
 'ons',
 'ja',
 'zal',
 'iets',
 'altijd',
 'moeten',
 '2',
 'onze',
 'u',
 'vandaag',
 'zich',
 'zelf',
 'staat',
 'ga',
 'andere',
 'zien',
 'omdat',
 'weet',
 'komen',
 'tijd

In [ ]:
char = CharLengthRatio("train", 0.8)
sentences_pair = char.get_ratio(sentences_pair)
sentences_pair

In [ ]:
word_length = WordLengthRatio("train", 0.8)
sentences_pair = word_length.get_ratio(sentences_pair)
sentences_pair

In [ ]:
levenshtein = LevenshteinRatio("train", 0.8)
sentences_pair = levenshtein.get_ratio(sentences_pair)
sentences_pair

In [ ]:
dependency = DependencyTreeDepthRatio("train", 0.8)
sentences_pair = dependency.get_ratio(sentences_pair)
sentences_pair

In [ ]:
word_rank = WordRankRatio("train", 0.8)
sentences_pair = word_rank.get_ratio(sentences_pair)
sentences_pair

Exploring features values from simpleText shared task

In [ ]:
# SIMPLETEXT_DATASET_PATH = DATASETS_PATH / "simpleText"

In [ ]:
WIKILARGE_DATASET_PATH = DATASETS_PATH / "wikilarge"

In [ ]:
complex_text = pd.read_csv(WIKILARGE_DATASET_PATH / "wikilarge.train.orig.txt", header=None, sep="\t", names=["original_text"])
simple_text = pd.read_csv(WIKILARGE_DATASET_PATH / "wikilarge.train.simp.txt", header=None, sep="\t",names=["simple_text"])

In [ ]:
# complex_text = pd.read_csv(SIMPLETEXT_DATASET_PATH / "simpleText.train.complex.txt", header=None, sep="\t", names=["original_text"])
# simple_text = pd.read_csv(SIMPLETEXT_DATASET_PATH / "simpleText.train.simple.txt", header=None, sep="\t",names=["simple_text"])

In [ ]:
sentences_pairs = pd.concat([complex_text, simple_text], axis=1)
sentences_pairs = sentences_pairs[1:60]

In [ ]:
results = []
for i,row in sentences_pairs.iterrows():
    sentences_pair = dict(original_text=row['original_text'], simple_text=row['simple_text'])
    sentences_pair = char.get_ratio(sentences_pair)
    sentences_pair = word_length.get_ratio(sentences_pair)
    sentences_pair = levenshtein.get_ratio(sentences_pair)
    sentences_pair = dependency.get_ratio(sentences_pair)
    sentences_pair = word_rank.get_ratio(sentences_pair)
    features = sentences_pair["original_text_preprocessed"].strip().split(" ")
    instance = dict(feature.split("_") for feature in features)
    results.append(instance)

In [ ]:
len(results)
results_df = pd.DataFrame(results).astype('float')

In [ ]:
results_df

In [ ]:
results_df

Summary statistics

In [ ]:
results_df.describe()

In [ ]:
results_df.describe()

Character Length Ratio

In [ ]:
fig = plt.figure(figsize=(10,6))
sns.distplot(results_df["CLR"])
fig.legend(labels=['Char Length'])
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,6))
sns.distplot(results_df["WLR"],label="Word Length")
fig.legend(labels=['Word Length'])
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,6))
sns.distplot(results_df["LR"])
fig.legend(labels=['Levenshtein'])
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,6))
sns.distplot(results_df["LR"])
fig.legend(labels=['Levenshtein'])
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,6))
sns.distplot(results_df["DTDR"])
fig.legend(labels=['Deep Tree'])
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,6))
sns.distplot(results_df["WRR"])
fig.legend(labels=['Word Rank'])
plt.show()